# **Event set creation for Risk Analysis**
This notebook explains how to create the input files for a risk analysis in the correct format and place them in the correct folder structure. For an explanation on how to derive the (compound) events and their (joint) probabilities, the reader is referred to the FloodAdapt Setup Guide [Event Set](../../../3_setup_guide/risk_analysis.qmd). 


## ⏱️ Step 1. Setup and Imports

In [ ]:
from datetime import datetime
from pathlib import Path

from flood_adapt import Settings, FloodAdapt
from flood_adapt import unit_system as us
from flood_adapt.config.sfincs import RiverModel
from flood_adapt.objects import (
    EventSet,
    SubEventModel,
    SyntheticEvent,
)
from flood_adapt.objects.forcing import (
    DischargeConstant,
    ForcingType,
    TimeFrame,
    ShapeType,
    TimeseriesFactory,
    RainfallSynthetic,
    SurgeModel,
    TideModel,
    WaterlevelSynthetic,
    WindConstant,
)

# Configure FloodAdapt
settings = Settings(
    DATABASE_ROOT=Path("../../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

fa = FloodAdapt(database_path=settings.database_path)

## 🗓️ Step 2. Set name, frequencies and parameters to build subevents
Set the name of the event set and the parameters for the input events and their frequencies. In this example, we are using three synthetic events with different storm surge heights and peak rainfall intensities. Alternatively, historic gauged and ungauged events or  hurricanes are possible as well, see [Events](../events/index.qmd) for examples on how to set up different subevents.

In [ ]:
name = "event_set"

# Peak surge values in the subevents. 
# Surge is timed to always peak at 24.8 hours after the event start, coinciding with high tide at a tidal phase of 0 and lasting for 10 hours.
surge_values = [2.5, 1, 3] # in meters

# Peak rainfall intensities in the three subevents. 
# Rainfall is applied as a 6 hour "block" shape preceding the peak of the storm surge.
rainfall_values = [10, 15, 5] # in mm/hr

# Occurrence frequencies of the three events
freq = [0.1, 2, 0.05]

## 🧩 Step 3. Create subevents
Create the subevents from the parameters defined in Step 1.

In [ ]:
events = []
number = 0
for surge, rainfall in zip(surge_values, rainfall_values):
    number += 1
    events.append(
        SyntheticEvent(
            name = f"subevent_{number}",
            time=TimeFrame(start_time=datetime(2020, 1, 1, 0, 0, 0),end_time=datetime(2020, 1, 3, 0, 0, 0)),
            forcings={
                # constant wind speed 10 m/, direction 90 degrees (from the East)
                ForcingType.WIND: [         
                    WindConstant(
                        speed=us.UnitfulVelocity(value=10, units=us.UnitTypesVelocity.mps), 
                        direction=us.UnitfulDirection(
                            value=90, units=us.UnitTypesDirection.degrees                  
                        ),
                    )
                ],
                # block rainfall starting 18h after event start, duration 6h, intensity 10 mm/hr 
                ForcingType.RAINFALL: [         
                    RainfallSynthetic(
                        timeseries = TimeseriesFactory.from_args(             
                            shape_type=ShapeType.block,
                            duration=us.UnitfulTime(
                                value=6, units=us.UnitTypesTime.hours
                            ),
                            peak_time=us.UnitfulTime(
                                value=21, units=us.UnitTypesTime.hours
                            ),
                            peak_value=us.UnitfulIntensity(
                                value=rainfall, units=us.UnitTypesIntensity.mm_hr
                            ),
                        )
                    )
                ],
                # constant discharge in Cooper River of 5000 cubic feet per second
                ForcingType.DISCHARGE: [         
                    DischargeConstant(
                        river=RiverModel(
                            name="cooper",
                            description="Cooper River",
                            x_coordinate=595546.3,
                            y_coordinate=3675590.6,
                            mean_discharge=us.UnitfulDischarge(
                                value=5000, units=us.UnitTypesDischarge.cfs
                            ),
                        ),
                        discharge=us.UnitfulDischarge(
                            value=5000, units=us.UnitTypesDischarge.cfs 
                        ),
                    )
                ],
                ForcingType.WATERLEVEL: [       
                    WaterlevelSynthetic(
                        # storm surge with gaussian shape, duration 10 hours, peak at 24h after event start_time, peak height 3m
                        surge=SurgeModel(       
                            timeseries=TimeseriesFactory.from_args(             
                                shape_type=ShapeType.gaussian,
                                duration=us.UnitfulTime(
                                    value=10, units=us.UnitTypesTime.hours
                                ),
                                peak_time=us.UnitfulTime(
                                    value=24, units=us.UnitTypesTime.hours
                                ),
                                peak_value=us.UnitfulLength(
                                    value=surge, units=us.UnitTypesLength.meters
                                ),
                            )
                        ),
                        # tide with 1m amplitude and 12.4h duration
                        tide=TideModel(     
                            harmonic_amplitude=us.UnitfulLength(
                                value=1, units=us.UnitTypesLength.meters
                            ),
                            harmonic_period=us.UnitfulTime(
                                value=12.4, units=us.UnitTypesTime.hours
                            ),
                            harmonic_phase=us.UnitfulTime(
                                value=0, units=us.UnitTypesTime.hours
                            ),
                        ),
                    )
                ],
            },
        )
    )

print(events)

## 💾 Step 4. Create and save event set

In this step, we create an event set from the `SubEvents` created in Step 2, load the actual `Event` objects into it, and save it to the database. This event set can then be used like any other event in a Scenario. Click [here](../scenarios/index.qmd) to see examples on how to build a scenario.

In [ ]:
sub_events = []
for event, frequency in zip(events, freq):
    sub_events.append(SubEventModel(name=event.name, frequency=frequency))


event_set = EventSet(
    name=name,
    sub_events=sub_events,
)
event_set.load_sub_events(events)

fa.save_event(event_set)
print(fa.get_events()["name"])